In [34]:
%set_env QXToken=YOUR_TOKEN

env: QXToken=YOUR_TOKEN


# 1

In [3]:
%pip install qiskit_ibm_runtime
from qiskit_ibm_runtime import (QiskitRuntimeService)

QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token="YOUR_TOKEN",
    set_as_default=True,
    # Use overwrite=True if you're updating your token.
    overwrite=True,
)
service = QiskitRuntimeService() 

In [17]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(channel = "ibm_quantum", token="YOUR_TOKEN",
                                  set_as_default=True, overwrite=True,)

service = QiskitRuntimeService()

In [18]:
BACKEND = service.backend(name="ibm_brisbane")

PATH_ANSWER = [0,1,2,3]

In [19]:
# grade part 1
from qc_grader.challenges.qgss_2024 import grade_lab2_ex1
grade_lab2_ex1(PATH_ANSWER, BACKEND)

Submitting your answer. Please wait...
Nice work, all your answers look correct!


# 2

In [20]:
LAYER_1_ANSWER = [(1,0),(3,2)]
LAYER_2_ANSWER = [(2,1)]

In [21]:
# grade part 2
from qc_grader.challenges.qgss_2024 import grade_lab2_ex2
grade_lab2_ex2(LAYER_1_ANSWER, LAYER_2_ANSWER, PATH_ANSWER, BACKEND)

Submitting your answer. Please wait...
Nice work, all your answers look correct!


# 3

In [22]:
%pip install MODULE_NAME

Note: you may need to restart the kernel to use updated packages.


In [26]:
from qiskit import QuantumCircuit
# Define the layers as provided
LAYER_1_ANSWER = [(1, 0), (3, 2)]
LAYER_2_ANSWER = [(2, 1)]

# Select a two-qubit Clifford gate from the backend's basis gates
GATE_NAME = 'cx'  # Assuming 'cx' (CNOT) is in backend.basis_gates

# Construct the first circuit
CIRC_1_ANSWER = QuantumCircuit(4)
for qubit1, qubit2 in LAYER_1_ANSWER:
    if GATE_NAME == 'cx':
        CIRC_1_ANSWER.cx(qubit1, qubit2)
    elif GATE_NAME == 'cz':
        CIRC_1_ANSWER.cz(qubit1, qubit2)
    elif GATE_NAME == 'ecr':
        CIRC_1_ANSWER.ecr(qubit1, qubit2)

# Construct the second circuit
CIRC_2_ANSWER = QuantumCircuit(4)
for qubit1, qubit2 in LAYER_2_ANSWER:
    if GATE_NAME == 'cx':
        CIRC_2_ANSWER.cx(qubit1, qubit2)
    elif GATE_NAME == 'cz':
        CIRC_2_ANSWER.cz(qubit1, qubit2)
    elif GATE_NAME == 'ecr':
        CIRC_2_ANSWER.ecr(qubit1, qubit2)

In [27]:
# grade part 3
from qc_grader.challenges.qgss_2024 import grade_lab2_ex3
grade_lab2_ex3(CIRC_1_ANSWER, LAYER_1_ANSWER, GATE_NAME, BACKEND)
grade_lab2_ex3(CIRC_2_ANSWER, LAYER_2_ANSWER, GATE_NAME, BACKEND)

Submitting your answer. Please wait...
Nice work, all your answers look correct!
Submitting your answer. Please wait...
Nice work, all your answers look correct!


In [28]:
from qiskit.circuit import Parameter
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

def eplg_circuit(num_qubits: int, depth: int, layer: QuantumCircuit, qubits: tuple[int, ...]) -> QuantumCircuit:
    if depth % 2 != 0:
        raise ValueError(f"The depth must be even, got {depth}")

    def parameters():
        _depth = 0
        while True:
            for zi in range(3):
                for q in range(num_qubits):
                    yield Parameter(f"d{_depth}_q{q}_z{zi}")
            _depth += 1
    _parameters = parameters()

    circ = QuantumCircuit(num_qubits)

    def _1q_layer():
        for j in range(3):
            for k in qubits:
                circ.rz(next(_parameters), k)
            if j == 2:
                continue
            circ.sx(qubits)

    for _depth in range(depth // 2):
        circ.barrier(qubits)
        _1q_layer()
        circ.barrier(qubits)
        circ.compose(layer, inplace=True)
    circ.barrier(qubits)
    _1q_layer()
    circ.barrier(qubits)
    
    # Assuming mirroring is fine
    circ = circ.compose(circ.inverse())

    circ.measure_active()

    pm = generate_preset_pass_manager(basis_gates=["ecr", "sx", "rz"], optimization_level=0)
    circ = pm.run(circ)

    circ.metadata["layer_depth"] = depth
    circ.metadata["qubits"] = qubits
    circ.metadata["num_qubits"] = num_qubits

    return circ

In [29]:
example_circuit_1 = eplg_circuit(
    num_qubits=BACKEND.num_qubits,
    depth=4,
    layer=CIRC_1_ANSWER,
    qubits=PATH_ANSWER,
)

In [30]:
example_circuit_2 = eplg_circuit(
    num_qubits=BACKEND.num_qubits,
    depth=4,
    layer=CIRC_2_ANSWER,
    qubits=PATH_ANSWER,
)

# 4

In [31]:
import numpy as np

from qiskit.primitives.containers.bindings_array import BindingsArray

def get_clifford_rz_samples(
    circ: QuantumCircuit,
    num_samples: int,
    rng: np.random.Generator,
) -> BindingsArray:
    """Uniformly sample the Rz parameters in a `QuantumCircuit`
    from -pi, -pi/2, 0, +pi/2, +pi.
    """

    _allowed_ops = {'rz', 'sx', 'sxdg', 'ecr', 'barrier', 'measure'}
    if (_other := set(circ.count_ops().keys()) - _allowed_ops):
        raise ValueError(f"Circuit must only contain ops: {_allowed_ops}, got {_other}")

    sampled_pars = rng.integers(-2, 3, size=(num_samples, len(circ.parameters))) * (np.pi/2)

    return BindingsArray({
        tuple(circ.parameters): sampled_pars
    })

In [32]:
num_samples = 10
ARRAY_1_ANSWER = get_clifford_rz_samples(
    circ=example_circuit_1,
    num_samples=num_samples,
    rng=np.random.default_rng(42))

In [33]:
# grade part 4
from qc_grader.challenges.qgss_2024 import grade_lab2_ex4
grade_lab2_ex4(ARRAY_1_ANSWER, example_circuit_1, num_samples)

Submitting your answer. Please wait...
Nice work, all your answers look correct! 
 Notice that the shape of the BindingsArray is [10], and there are {num_parameters} parameters.
